In [ ]:
from os import getenv
from aleph_alpha_client import Client
from intelligence_layer.classify import SingleLabelClassify, ClassifyInput, ClassifyOutput
from pprint import pprint

client = Client(getenv("AA_API_TOKEN"))
task = SingleLabelClassify(client)
input = ClassifyInput(text="The weather is very nice.", labels=["Weather", "Traffic"])

output = task.run(input)
pprint(output.debug_log.filter("debug").model_dump())

